# Extract the respiration signals from a video

This notebook extracts the raw signals for each fine-tuned EfficientPhys models.

In [ ]:
import os

import numpy as np

models_dirs = [
    # Trained 5 epochs with MSE loss
    os.path.join('..', 'models', 'efficeint_phys_fine_tuned', '20240503_174905'),
]

In [ ]:
import respiration.utils as utils

device = utils.get_torch_device()
dim = 72
frame_depth = 20

In [ ]:
model_info = []

for model_dir in models_dirs:
    manifest_path = os.path.join(model_dir, 'manifest.json')
    manifest = utils.read_json(manifest_path)

    for tuned_model in manifest['tuned_models']:
        model_info.append({
            'id': manifest['id'],
            'base_model': manifest['base_model'],
            'loss_fn': manifest['loss_fn'],
            'model': tuned_model['model'],
            'epoch': tuned_model['epoch'],
            'validation_loss': tuned_model['validation_loss'],
        })

In [ ]:
import pandas as pd

model_df = pd.DataFrame(model_info)
model_df

In [ ]:
import torch
from respiration.extractor.efficient_phys import EfficientPhys


def load_model(path) -> EfficientPhys:
    # Wrap modul in nn.DataParallel
    model = EfficientPhys(img_size=dim, frame_depth=frame_depth)
    # Fix model loading: Some key have an extra 'module.' prefix
    model = torch.nn.DataParallel(model)
    model.to(device)
    model.load_state_dict(torch.load(path, map_location=device))

    _ = model.eval()

    return model

In [ ]:
import respiration.dataset as repository

dataset = repository.from_default()

In [ ]:
def predict_signal(model: EfficientPhys, subject: str, setting: str) -> tuple[np.ndarray, int]:
    video_path = dataset.get_video_path(subject, setting)
    frame_count = utils.get_frame_count(video_path)

    chunk_size = (frame_count // frame_depth) * frame_depth - (frame_depth - 1)
    # chunk_size = frame_depth * 20 + 1

    frames, meta = utils.read_video_rgb(video_path, chunk_size)
    frames = utils.down_sample_video(frames, dim)

    frames = torch.tensor(frames, dtype=torch.float32, device=device).permute(0, 3, 1, 2)

    with torch.no_grad():
        prediction = model(frames).cpu().detach().numpy().squeeze()

    return prediction, chunk_size

In [ ]:
predictions = []

for model_dir in models_dirs:
    manifest_path = os.path.join(model_dir, 'manifest.json')
    manifest = utils.read_json(manifest_path)

    best_model = manifest['tuned_models'][-1]
    model_path = os.path.join(model_dir, best_model['model'])

    model = load_model(model_path)

    for scenario in manifest['testing_scenarios']:
        if not dataset.contains(scenario[0], scenario[1]):
            print(f"Skipping {scenario[0]} - {scenario[1]}")
            continue

        print(f"Processing {scenario[0]} - {scenario[1]}")
        prediction, chunks = predict_signal(model, scenario[0], scenario[1])

        predictions.append({
            'id': manifest['id'],
            'subject': scenario[0],
            'setting': scenario[1],
            'chunk_size': chunks,
            'prediction': prediction,
        })

In [ ]:
prediction_df = pd.DataFrame(predictions)

evaluation_dir = os.path.join('..', 'evaluation', 'efficeint_phys_fine_tuned')
if not os.path.exists(evaluation_dir):
    os.makedirs(evaluation_dir)

evaluation_path = os.path.join(evaluation_dir, 'predictions.csv')
prediction_df.to_csv(evaluation_path, index=False)

prediction_df